In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
from supabase import create_client, Client
import os 
import psycopg2

############# SUPABASE Creation #########
db_url = os.getenv('DATABASE_URL')
conn = psycopg2.connect(db_url)

cur = conn.cursor()
query = '''
        INSERT INTO test1 (
            id, name,age
        ) VALUES (1, 'Johnathan', 29);
'''

cur.execute(query)

conn.commit()
cur.close()
conn.close()    


In [2]:
import fitz
from langchain_core.documents import Document
from PIL import Image
import torch
import numpy as np
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from sklearn.metrics.pairwise import cosine_similarity
import os
import base64
import io
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from transformers import CLIPProcessor, CLIPModel

/Users/jeremiahtay/mental_health_agentic_chatbot/chatbot_proj/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def embed_image(image_data):
    """Embed image using CLIP model"""
    if isinstance(image_data, str):
        image = Image.open(image_data).convert("RGB")
    else:
        image = image_data
    
    inputs = clip_processor(images = image, return_tensors = "pt")
    with torch.no_grad():
        features = clip_model.get_image_features(**inputs)
        # Normalize embeddings to unit vector
        features = features / features.norm(dim = -1, keepdim = True)
        return features.squeeze().numpy()


def embed_text(text):
    """Embed text using CLIP model"""
    inputs = clip_processor(
        text = text,
        return_tensors = "pt",
        padding = True,
        truncation = True,
        max_length = 77
    )

    with torch.no_grad():
        features = clip_model.get_text_features(**inputs)
        # Normalize embeddings to unit vector
        features = features / features.norm(dim = -1, keepdim = True)
        return features.squeeze().numpy()

## Compiling PDF files to folder

In [13]:
import requests
import re
urls = [
    "https://www.nimh.nih.gov/health/publications/my-mental-health-do-i-need-help",
    "https://www.nimh.nih.gov/health/publications/panic-disorder-when-fear-overwhelms",
    "https://www.nimh.nih.gov/health/publications/pandas",
    "https://www.nimh.nih.gov/health/publications/phobias-and-phobia-related-disorders",
    "https://www.nimh.nih.gov/health/publications/social-anxiety-disorder-more-than-just-shyness"
]

pdf_urls = [
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/5-action-steps-help-someone-having-thoughts-suicide.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/adhd-in-adults-four-things-to-know.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/adhd-2024.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/autism-spectrum-disorder/autism-spectrum-disorder.pdf",
    "https://www.nimh.nih.gov/sites/default/files/health/publications/bipolar-disorder/bipolar-disorder.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/bipolar-disorder-in-children-and-teens/bipolar-disorder-in-children-and-teens.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/bipolar-disorder-in-children-and-teens/bipolar-disorder-in-teens-and-young-adults.pdf",
    "https://www.nimh.nih.gov/health/publications/borderline-personality-disorder",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/children-and-mental-health/children-and-mental-health.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/clinical-research-trials-and-you-questions-and-answers/clinical-research-trials-you-questions-answers.pdf",
    "https://www.nimh.nih.gov/sites/default/files/health/publications/depression/depression.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/depression-in-women/depression-women_508.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/disruptive-mood-dysregulation-disorder/disruptive-mood-dysregulation-disorder-the-basics_1.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/eating-disorders/eating-disorders-what-you-need-to-know.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/suicide-faq/frequently-asked-questions-about-suicide.pdf",
    "https://www.nimh.nih.gov/health/publications/generalized-anxiety-disorder-gad",
    "https://www.nimh.nih.gov/health/publications/what-is-telemental-health",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/helping-children-and-adolescents-cope-with-disasters-and-other-traumatic-events/helping-children-and-adolescents-cope-with-traumatic-events.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/so-stressed-out-fact-sheet/Im-So-Stressed-Out.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/so-stressed-out-infographic/so-stressed-out-infographic.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/lets-talk-about-eating-disorders/lets-talk-about-eating-disorders.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/looking-at-my-genes/looking-at-my-genes-what-can-they-tell-me-about-my-mental-health.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/obsessive-compulsive-disorder-when-unwanted-thoughts-take-over/obsessive-compulsive-disorder-508.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/perinatal-depression.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/post-traumatic-stress-disorder-ptsd/post-traumatic-stress-disorder_1.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/schizophrenia/schizophrenia_1.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/seasonal-affective-disorder/seasonal-affective-disorder-508.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/seasonal-affective-disorder/seasonal_affective_disorder_more_than_the_winter_blues.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/teen-depression/Teen_Depression_More_Than_Just_Moodiness_2022.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/the-teen-brain-7-things-to-know/teen-brain-7-things-to-know.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/tips-for-talking-with-your-health-care-provider/tips-for-talking-with-a-health-care-provider-about-your-mental-health_1.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/understanding-psychosis/23-MH-8110-Understanding-Psychosis.pdf",
    "https://www.nimh.nih.gov/sites/default/files/health/publications/chronic-illness-mental-health/understanding-link-between-chronic-disease-depression.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/warning-signs-of-suicide/warning-signs-of-suicide.pdf",
    "https://www.nimh.nih.gov/sites/default/files/documents/health/publications/lets-talk-about-eating-disorders/common-types-eating-disorders.pdf"
]  # Replace with the actual PDF URL

for pdf_url in pdf_urls:
    output_filename = pdf_url.split('/')[-1]
    file_path = os.path.join("pdf", output_filename)
    try:
        response = requests.get(pdf_url)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

        with open(file_path, "wb") as f:
            f.write(response.content)
        print(f"PDF downloaded successfully to {output_filename}")

    except requests.exceptions.RequestException as e:
        print(f"Error downloading PDF: {e}")

PDF downloaded successfully to 5-action-steps-help-someone-having-thoughts-suicide.pdf
PDF downloaded successfully to adhd-in-adults-four-things-to-know.pdf
PDF downloaded successfully to adhd-2024.pdf
PDF downloaded successfully to autism-spectrum-disorder.pdf
PDF downloaded successfully to bipolar-disorder.pdf
PDF downloaded successfully to bipolar-disorder-in-children-and-teens.pdf
PDF downloaded successfully to bipolar-disorder-in-teens-and-young-adults.pdf
PDF downloaded successfully to borderline-personality-disorder
PDF downloaded successfully to children-and-mental-health.pdf
PDF downloaded successfully to clinical-research-trials-you-questions-answers.pdf
PDF downloaded successfully to depression.pdf
PDF downloaded successfully to depression-women_508.pdf
PDF downloaded successfully to disruptive-mood-dysregulation-disorder-the-basics_1.pdf
PDF downloaded successfully to eating-disorders-what-you-need-to-know.pdf
PDF downloaded successfully to frequently-asked-questions-about-

## Chunking PDF files to Postgres DB

In [ ]:
import os
import json
import fitz  # PyMuPDF
import psycopg2
import requests
from bs4 import BeautifulSoup
from psycopg2.extras import execute_values
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

def chunk_pdf(pdf_path: str) -> list[Document]:
    """
    Opens a PDF, extracts its text, and splits it into processed chunks.

    Args:
        pdf_path (str): The file path to the PDF document.

    Returns:
        list[Document]: A list of LangChain Document objects, each representing a
                        chunk of text with populated metadata.
    """
    doc = fitz.open(pdf_path)
    splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=200)

    # Extract all text from the PDF into a single string
    full_text = ""
    for page in doc:
        full_text += page.get_text() + "\n\n"
    
    doc.close() # Close the document after extracting text

    # Create a single Document object for the entire text
    doc_for_splitting = Document(page_content = full_text)

    # 3. Split the document into chunks
    chunks = splitter.split_documents([doc_for_splitting])

    # 4. Add metadata to each chunk
    for i, chunk in enumerate(chunks):
        chunk.metadata["source"] = pdf_path
        chunk.metadata["chunk_number"] = i + 1

    return chunks


def insert_chunk_to_db(pdf_path: str, db_url: str):
    doc_chunks = chunk_pdf(pdf_path)
    data_to_insert = []

    for chunk in doc_chunks:
        content = chunk.page_content
        metadata = chunk.metadata
        source = metadata.get('source')
        chunk_number = metadata.get('chunk_number')

        data_to_insert.append((
            json.dumps(metadata),
            source,
            chunk_number,
            content
        ))

    try:
        with psycopg2.connect(db_url) as conn:
            with conn.cursor() as cur:
                # Define the INSERT statement
                query = "INSERT INTO rag_text_chunks (metadata, source, chunk_number, content) VALUES %s"

                # Execute the batch insert
                execute_values(cur, query, data_to_insert)
            
            print(f"Successfully inserted {len(data_to_insert)} chunks from {os.path.basename(pdf_path)}.")

    except psycopg2.Error as e:
        print(f"Database error: {e}")

if __name__ == "__main__":
    pdf_directory = "pdf"
    pdf_path = "pdf/5-action-steps-help-someone-having-thoughts-suicide.pdf"
    db_url = os.getenv('DATABASE_URL')
    
    if not db_url:
        print("Error: DATABASE_URL environment variable not set.")
    elif not os.path.isdir(pdf_directory):
        print(f"Error: Directory '{pdf_directory}' not found.")
    else:
        print(f"Starting to process PDF files in '{pdf_directory}' directory...")
        # Loop through all files in the specified directory
        for filename in os.listdir(pdf_directory):
            # Check if the file has a .pdf extension (case-insensitive)
            if filename.lower().endswith(".pdf"):
                # Construct the full path to the PDF file
                full_pdf_path = os.path.join(pdf_directory, filename)
                
                # Call the function to process and insert this single PDF
                insert_chunk_to_db(full_pdf_path, db_url)
        print("Processing complete.")


Starting to process PDF files in 'pdf' directory...
Successfully inserted 6 chunks from My Mental Health_ Do I Need Help_ - National Institute of Mental Health (NIMH).pdf.
Successfully inserted 26 chunks from children-and-mental-health.pdf.
Successfully inserted 20 chunks from clinical-research-trials-you-questions-answers.pdf.
Successfully inserted 27 chunks from Panic Disorder_ What You Need to Know - National Institute of Mental Health (NIMH).pdf.
Successfully inserted 14 chunks from Phobias and Phobia-Related Disorders - National Institute of Mental Health (NIMH).pdf.
Successfully inserted 23 chunks from eating-disorders-what-you-need-to-know.pdf.
Successfully inserted 8 chunks from teen-brain-7-things-to-know.pdf.
Successfully inserted 23 chunks from frequently-asked-questions-about-suicide.pdf.
Successfully inserted 19 chunks from adhd-in-adults-four-things-to-know.pdf.
Successfully inserted 37 chunks from bipolar-disorder.pdf.
Successfully inserted 30 chunks from perinatal-depre

## Initialising ClipModel

In [28]:
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model.eval()

splitter = RecursiveCharacterTextSplitter(chunk_size = 700, chunk_overlap = 200)

Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 20971.52it/s]


In [39]:
# --- Step 1: Extract text from all pages into one string ---
full_text = ""
for page in doc:
    # Add a separator to mark page breaks, which helps the splitter
    full_text += page.get_text() + "\n\n"

# --- Step 2: Create a single Document from the full text ---
doc_for_splitting = Document(page_content=full_text)

# --- Step 3: Split the entire document at once ---
final_chunks = splitter.split_documents([doc_for_splitting])

for i, chunk in enumerate(final_chunks):
    chunk.metadata["source"] = pdf_path
    chunk.metadata["chunk_number"] = i



# Now you have a single list `final_chunks` that you can process
print(f"Total chunks created from the whole document: {len(final_chunks)}")
print("\n--- Example of the first chunk ---")
print(final_chunks[0].page_content)

# You can now loop through `final_chunks` to create your embeddings
all_docs = final_chunks
all_embeddings = [embed_text(chunk.page_content) for chunk in final_chunks]

Total chunks created from the whole document: 3

--- Example of the first chunk ---
5
Action Steps to Help Someone 
Having Thoughts of Suicide
We can all take steps to help prevent suicide. Knowing the warning signs 
for suicide and how to get help can save lives.
Here are 5 steps you can take to #BeThe1To help someone who is having 
thoughts of suicide: 
1. ASK:
“Are you thinking about suicide?” It’s not an easy question to ask, but it can help 
start a conversation. Studies show that asking people if they are suicidal does not 
increase suicidal behavior or thoughts. 
2. BE THERE: 
Listening without judgment is key to learning what the person is thinking and 
feeling. Research suggests acknowledging and talking about suicide may reduce 
suicidal thoughts.
